Final Project

Explanation of Motivation & Provide Resources

Motivation: each tutorial should be sufficiently motivated. If there is not motivation for the analysis, why would we ‘do data science’ on this topic?

Resources: tutorials should help the reader learn a skill, but they should also provide a launching pad for the reader to further develop that skill. The tutorial should link to additional resources wherever appropriate, so that a well-motivated reader can read further on techniques that have been used in the tutorial.

Phase 1: Data Collection, Curation, and Parsing

In [1]:
# Import Libraries
import pandas as pd
import numpy as np

print("Beginning of project")

Beginning of project


Phase 2: Data Management and Representation

Phase 3: Exploratory Data Analysis

Phase 4: Hypothesis Testing

Phase 5: Communication of Insights Attained

Understanding: the reader of the tutorial should walk away with some new understanding of the topic at hand. If it’s not possible for a reader to state ‘what they learned’ from reading your tutorial, then why do the analysis?